In [2]:
import pandas as pd

data = pd.read_csv('all_glucose_prediction_results.csv')
data

Category  Start Date  Previous Lower Bound  Actual Glucose Change  \
0          Lows  2023-11-02              0.358292               0.547059   
1          Lows  2023-11-03              0.328162               0.506667   
2          Lows  2023-11-04              0.263143               0.517647   
3          Lows  2023-11-07              0.307353               0.550000   
4          Lows  2023-11-10              0.315588               0.795000   
..          ...         ...                   ...                    ...   
165  Very Highs  2023-12-30              0.367664               0.728947   
166  Very Highs  2024-01-01              0.255681               0.794167   
167  Very Highs  2024-01-02              0.353486               0.817500   
168  Very Highs  2024-01-04              0.403305               0.844167   
169  Very Highs  2024-01-05              0.190299               0.850833   

     Previous Upper Bound  Actual Variance  Prediction Status  \
0                0.804208         0.218897               True   
1                0.765956         0.243524               True   
2                0.750190         0.210294               True   
3                0.727941         0.234412               True   
4                0.784412         0.734184              False   
..                    ...              ...                ...   
165              1.301902         0.473267               True   
166              1.202214         0.440681               True   
167              1.234847         0.414195               True   
168              1.231695         0.653868               True   
169              1.498034         0.645248               True   

     Predicted Glucose Change  Predicted Variance  
0                    0.581250            0.222958  
1                    0.547059            0.218897  
2                    0.506667            0.243524  
3                    0.517647            0.210294  
4                    0.550000            0.234412  
..                        ...                 ...  
165                  0.834783            0.467119  
166                  0.728947            0.473267  
167                  0.794167            0.440681  
168                  0.817500            0.414195  
169                  0.844167            0.653868  

[170 rows x 9 columns]

In [3]:
category_stats = data.groupby('Category')[['Actual Glucose Change', 'Actual Variance']].agg(
    ['mean', 'median']
).reset_index()

category_order = ["Lows", "Mediums", "Highs", "Very Highs"]

category_stats_ordered = category_stats.set_index('Category').loc[category_order].reset_index()

print("Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):")
print(category_stats_ordered)

Category Statistics in Custom Order (Lows, Mediums, Highs, Very Highs):
     Category Actual Glucose Change           Actual Variance          
                               mean    median            mean    median
0        Lows              0.711144  0.719231        0.504292  0.568423
1     Mediums              0.592845  0.539047        0.197642  0.134827
2       Highs              0.774556  0.764759        0.414866  0.437251
3  Very Highs              0.891606  0.850000        0.459572  0.486605


In [4]:
data_false_predictions = data[data['Prediction Status'] == False].copy()

if not data_false_predictions.empty:
    data_false_predictions.loc[:, 'Absolute Error Glucose Change'] = abs(data_false_predictions['Actual Glucose Change'] - data_false_predictions['Predicted Glucose Change'])
    data_false_predictions.loc[:, 'Absolute Error Variance Change'] = abs(data_false_predictions['Actual Variance'] - data_false_predictions['Predicted Variance'])

    mae_stats_glucose = data_false_predictions.groupby('Category')['Absolute Error Glucose Change'].agg(['mean', 'median']).reset_index()
    mae_stats_variance = data_false_predictions.groupby('Category')['Absolute Error Variance Change'].agg(['mean', 'median']).reset_index()

    available_categories_glucose = [cat for cat in category_order if cat in mae_stats_glucose['Category'].values]
    mae_stats_glucose_ordered = mae_stats_glucose.set_index('Category').loc[available_categories_glucose].reset_index()

    available_categories_variance = [cat for cat in category_order if cat in mae_stats_variance['Category'].values]
    mae_stats_variance_ordered = mae_stats_variance.set_index('Category').loc[available_categories_variance].reset_index()

    print("MAE Statistics for Glucose Change (False Predictions):")
    print(mae_stats_glucose_ordered)
    
    print("\nMAE Statistics for Variance Change (False Predictions):")
    print(mae_stats_variance_ordered)
else:
    print("No entries found with Prediction Status as False.")

MAE Statistics for Glucose Change (False Predictions):
  Category      mean    median
0     Lows  0.298991  0.298991
1  Mediums  0.164286  0.164286
2    Highs  0.217874  0.217874

MAE Statistics for Variance Change (False Predictions):
  Category      mean    median
0     Lows  0.547799  0.547799
1  Mediums  0.259286  0.259286
2    Highs  0.423745  0.423745
